# Système de recommandation

In [13]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [20]:
df_titre = pd.read_csv("df_film_filtre.csv").drop(columns = "Unnamed: 0")

In [22]:
df = pd.read_csv("df_film_filtre.csv").drop(columns = ["Unnamed: 0", "poster_path", "titre"])

In [23]:
df.head()

,année,temps_minutes,note,nb_votes,Real,popularity,production_countries,Genre1,Genre2,Genre3
0,1917,50.0,5.1,26,Alice Guy,1.341,['US'],Drama,NaN,NaN
1,1918,50.0,6.6,24,Alice Guy,0.840,['US'],Drama,NaN,NaN
2,1897,100.0,5.2,538,Enoch J. Rector,0.958,['US'],Documentary,News,Sport
3,1922,70.0,6.6,49,J. Stuart Blackton,0.841,['GB'],Adventure,History,NaN
4,1933,55.0,7.2,46,J. Stuart Blackton,0.872,['GB'],History,NaN,NaN


## Séparer les colonnes num et cat

Non supervié

In [24]:
col_num = df.select_dtypes('number').columns
col_cat = df.select_dtypes('object').columns

C'est l'étape qui va transformer les colonnes numériques (standardisation etc)

In [25]:
numeric_transformer = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy = "most_frequent")), #Remplace les valeurs manquantes
    ("scaling", StandardScaler()) #Standardise 
])

In [26]:
categorical_transformer =  Pipeline(steps = [
    ("imputer", SimpleImputer(strategy = "most_frequent")), #Remplace les valeurs manquantes
    ("encoder", OneHotEncoder(handle_unknown= "ignore"))
])

On an cat et num, le porcesseur réunit les 2 

In [27]:
#processeur
preprocessor = ColumnTransformer(
    transformers= [("num", numeric_transformer, col_num),
                   ("cat", categorical_transformer, col_cat)
                   ]
)

In [77]:
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor), 
    ("model", NearestNeighbors(n_neighbors= 6, metric = 'euclidean')) # Le modèle qu'on veut entraîner 
]
)

In [78]:
pipeline.fit(df)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scaling',
                                                                   StandardScaler())]),
                                                  Index(['année', 'temps_minutes', 'note', 'nb_votes', 'popularity'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Real', 'production_countries', 'Genre1', 'Genre2', 'Genre3'], dtype='object'))])),
                ('model', NearestNeighbors(metric='euclidean', n_neighbors=6))])

In [79]:
data = pipeline.named_steps["preprocessor"].transform(df_titre.iloc[[45176]]) #Testé avec la première ligne [[pour aveoir un df pas une serie]]

In [80]:
dis,index = pipeline.named_steps["model"].kneighbors(data)

In [81]:
for titre in index[0][1:]:
    display(df_titre.iloc[[titre]])

,titre,année,temps_minutes,note,nb_votes,Real,popularity,poster_path,production_countries,Genre1,Genre2,Genre3
45177,The Lord of the Rings: The Return of the King,2003,201.0,9.0,2025818,Peter Jackson,65.571,/rCzpDGLbOoPwLjy3OAm5NUPOTrC.jpg,"['NZ', 'US']",Action,Adventure,Drama


,titre,année,temps_minutes,note,nb_votes,Real,popularity,poster_path,production_countries,Genre1,Genre2,Genre3
26180,The Godfather,1972,175.0,9.2,2063067,Francis Ford Coppola,113.216,/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,['US'],Crime,Drama,NaN


,titre,année,temps_minutes,note,nb_votes,Real,popularity,poster_path,production_countries,Genre1,Genre2,Genre3
45178,The Lord of the Rings: The Two Towers,2002,179.0,8.8,1826369,Peter Jackson,62.983,/5VTN0pR8gcqV3EPUHHfMGnJYN9L.jpg,"['NZ', 'US']",Action,Adventure,Drama


,titre,année,temps_minutes,note,nb_votes,Real,popularity,poster_path,production_countries,Genre1,Genre2,Genre3
59202,Interstellar,2014,169.0,8.7,2188340,Christopher Nolan,121.151,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,"['GB', 'US']",Adventure,Drama,Sci-Fi


,titre,année,temps_minutes,note,nb_votes,Real,popularity,poster_path,production_countries,Genre1,Genre2,Genre3
59203,The Dark Knight Rises,2012,164.0,8.4,1868531,Christopher Nolan,66.002,/85cWkCVftiVs0BVey6pxX8uNmLt.jpg,['US'],Action,Drama,Thriller
